In [140]:
import numpy as np
import csv
import plotly
import plotly.graph_objs as go
from PIL import Image, ImageDraw, ImageFont
from scipy.ndimage import gaussian_filter

plotly.offline.init_notebook_mode(connected=True)
py = plotly.offline

# read data

def convert(row):
    id = row[0]
    rssi = row[1]
    x = row[2]
    y = row[3]
    return (id, int(rssi), (int(x) + 390) / 3, (int(y) + 960) / 3)
    
with open('rssi_god.csv', 'rb') as f:
    reader = csv.reader(f)
    data = map(convert, reader)
    
# global coordinates to pixels conversion

w9_pixel_origin = np.array([252, 1012])
w9_global_origin = np.array([55.94444938963575,-3.1869836524128914])

def get_conversion_data():
    se1_pixel = np.array([409, 479])
    nw10_pixel = np.array([161, 897])

    se1_global = np.array([55.9444578385393,-3.1866151839494705])
    nw10_global = np.array([55.94449107087541,-3.186941407620907])

    se1_pixel_trans = se1_pixel - w9_pixel_origin
    nw10_pixel_trans = nw10_pixel - w9_pixel_origin

    se1_global_trans = se1_global - w9_global_origin
    nw10_global_trans = nw10_global - w9_global_origin

    return ((se1_pixel_trans, nw10_pixel_trans), (se1_global_trans, nw10_global_trans))

coor_change = np.array([[-2.67731e6, 487478.], [-1.32371e6, -1.41618e6]])

# convert global coordinates to pixels

beacons_global = np.array([
    [55.9444578385393,-3.1866151839494705],
    [55.94444244275808,-3.18672649562358860],
    [55.94452336441765,-3.1866540759801865],
    [55.94452261340533,-3.1867526471614838],
    [55.94448393625199,-3.1868280842900276],
    [55.94449050761571,-3.1866483762860294],
    [55.94443774892113,-3.1867992505431175],
    [55.944432116316044,-3.186904862523079],
    [55.94444938963575,-3.1869836524128914],
    [55.94449107087541,-3.186941407620907]
])

beacons_pixel = np.array(np.transpose(coor_change.dot(np.transpose(beacons_global - w9_global_origin))) + w9_pixel_origin, dtype=np.int16)

# beacon id

beacon_ids = ['cd', 'd7', '17', '51', '43', 'b8', '2a', 'f8', '3d', '62',
            '3']

# beacon readings

data_for_beacon = {}
for id in beacon_ids:
    data_for_beacon[id] = []
    
for row in data:
    beacon = row[0]
    data_for_beacon[beacon].append(row)
    
# drawing images
    
def draw_image(points, b):
    # get an image
    img = Image.open('map.png')
    fnt = ImageFont.truetype('/Users/piotr/Library/Fonts/Literation Mono Powerline.ttf', 40)
    
    d = ImageDraw.Draw(img)

    d.text((10,10), b, font=fnt, fill=(0,0,0,0))

    for (ident, beacon) in zip(beacon_ids, beacons_pixel):
        (x, y) = beacon
        d.text((x+10,y+10), ident, font=fnt, fill=(0,0,0,0))
        d.ellipse([x, y, x+5, y+5], outline=(0,255,0,0), fill=(0,255,0,0))

    for point in points:
        x = point[0]
        y = point[1]
        d.ellipse([x, y, x+5, y+5], outline=(255,0,0,0), fill=(255,0,0,0))

    img.show(title=b)
    
def draw_image_for(b_id):
    points = np.array([[row[2], row[3]] for row in data_for_beacon[b_id]], dtype=np.int32)
    draw_image(points, b_id)
    
# analytics

def rssi_from_data(data):
    return np.array([row[1] for row in data], dtype=np.int16)

def pos_from_data(data):
    return np.array([[row[2], row[3]] for row in data], dtype=np.int16)

def dist_from_point(point, positions):
    diff = positions - point
    return np.linalg.norm(diff, axis=1)
    
def beacon_fun(b_id, b_pos):
    data = data_for_beacon[b_id]
    pos = pos_from_data(data)
    rssi = rssi_from_data(data)
    dist = dist_from_point(b_pos, pos)
    return (pos, rssi, dist)

def map_strongest_signal(rssi, pos, n, b_id):
    largest = np.argsort(rssi)[:n]
    points = pos[largest]
    draw_image(points, b_id)
    
def plot_rssi_f_dist(b_id, b_pos):
    (beacon_pos, beacon_rssi, dist) = beacon_fun(b_id, b_pos)
    py.iplot({
        "data": [go.Scatter(x=dist, y=beacon_rssi, mode='markers')],
        "layout": go.Layout(title=b_id)
    })
    
def normalize(array):
    return (array - np.mean(array)) / np.std(array)

In [141]:
# plot rssi as function of distance from beacon

for (b_id, b_pos) in zip(beacon_ids, beacons_pixel):
    if data_for_beacon[b_id]:
        plot_rssi_f_dist(b_id, b_pos)

In [139]:
# now for each beacon readings serially

def rssi_changes(b_id, b_pos):
    data = data_for_beacon[b_id]
    rssi = rssi_from_data(data)
    dist = dist_from_point(b_pos, pos_from_data(data))
    normalized_rssi = normalize(rssi)
    normalized_dist = normalize(dist)
    filtered_rssi = gaussian_filter(normalized_rssi, sigma=3)
    n = len(rssi)
    datalines = ['normalized_rssi', 'normalized_dist', 'filtered_rssi']

    py.iplot({
        'data': [go.Scatter(name=data, x=np.linspace(0, 100, n), y=locals()[data]) for data in datalines],
        'layout': go.Layout(title=b_id)
    })

for (b_id, b_pos) in zip(beacon_ids, beacons_pixel):
    if len(data_for_beacon[b_id]) > 20:
        rssi_changes(b_id, b_pos)

In [142]:
# draw for all beacons

for b in beacon_ids:
    draw_image_for(b)

In [63]:
# more analytics

draw_image_for('17')

rssi = np.fromiter((row[1] for row in data), dtype=np.int16)
py.offline.iplot([go.Histogram(x=rssi)])

(beacon_b8_pos, beacon_b8_rssi, dist_b8) = beacon_fun('b8', np.array([161, 897]))
py.iplot({
    "data": [go.Scatter(x=dist_b8, y=beacon_b8_rssi, mode='markers')],
    "layout": go.Layout(title="hello world")
})

(beacon_17_pos, beacon_17_rssi, dist_17) = beacon_fun('17', np.array([356, 922]))

py.iplot({
    "data": [go.Scatter(x=dist_17, y=beacon_17_rssi, mode='markers')],
    "layout": go.Layout(title="hello world")
})

map_strongest_signal(beacon_b8_rssi, beacon_b8_pos, 50, 'b8')
map_strongest_signal(beacon_17_rssi, beacon_17_pos, 50, '17')

okay now look at all signals for a beacon and plot signal strength as function of distance

TODO cutoff at -90, take average (arithm/geom/harmonic) or median of rssi, and then plot on the map with different transparency

TODO triangulation with gaussians

In [ ]:
import math
import numpy as np
from numpy.random import randn

def compute_dog_data(z_var, process_var, count=1, dt=1.):
    "returns track, measurements 1D ndarrays"
    x, vel = 0., 1.
    z_std = math.sqrt(z_var) 
    p_std = math.sqrt(process_var)
    xs, zs = [], []
    for _ in range(count):
        v = vel + (randn() * p_std)
        x += v*dt        
        xs.append(x)
        zs.append(x + randn() * z_std)        
    return np.array(xs), np.array(zs)

In [ ]:
from filterpy.stats import plot_covariance_ellipse

dt = 0.3
F = np.array([[1, dt], [0, 1]])
x = np.array([10.0, 4.5])
P = np.diag([500, 500])
plot_covariance_ellipse(x, P, edgecolor='r')
x, P = predict(x, P, F, Q=0)
plot_covariance_ellipse(x, P, edgecolor='k', ls='dashed')

In [ ]:
from filterpy.common import Q_discrete_white_noise
Q = Q_discrete_white_noise(dim=2, dt=1., var=2.35)
print(Q)

In [ ]:
H = np.array([[1., 0.]])
R = np.array([[5.]])

In [ ]:
from filterpy.kalman import KalmanFilter
from filterpy.common import Q_discrete_white_noise

def pos_vel_filter(x, P, R, Q=0., dt=1.0):
    """ Returns a KalmanFilter which implements a
    constant velocity model for a state [x dx].T
    """
    
    kf = KalmanFilter(dim_x=2, dim_z=1)
    kf.x = np.array([x[0], x[1]]) # location and velocity
    kf.F = np.array([[1., dt],
                     [0.,  1.]])  # state transition matrix
    kf.H = np.array([[1., 0]])    # Measurement function
    kf.R *= R                     # measurement uncertainty
    if np.isscalar(P):
        kf.P *= P                 # covariance matrix 
    else:
        kf.P[:] = P               # [:] makes deep copy
    if np.isscalar(Q):
        kf.Q = Q_discrete_white_noise(dim=2, dt=dt, var=Q)
    else:
        kf.Q[:] = Q
    return kf

In [ ]:
dt = .1
x = np.array([0., 0.]) 
kf = pos_vel_filter(x, P=500, R=5, Q=0.1, dt=dt)

In [ ]:
# from kf_book.mkf_internal import plot_track

def run(x0=(0.,0.), P=500, R=0, Q=0, dt=1.0, 
        track=None, zs=None,
        count=0, do_plot=True, **kwargs):
    """
    track is the actual position of the dog, zs are the 
    corresponding measurements. 
    """

    # Simulate dog if no data provided. 
    if zs is None:
        track, zs = compute_dog_data(R, Q, count)

    # create the Kalman filter
    kf = pos_vel_filter(x0, R=R, P=P, Q=Q, dt=dt)  

    # run the kalman filter and store the results
    xs, cov = [], []
    for z in zs:
        kf.predict()
        kf.update(z)
        xs.append(kf.x)
        cov.append(kf.P)

    xs, cov = np.array(xs), np.array(cov)
#     if do_plot:
#         plot_track(xs[:, 0], track, zs, cov, 
#                    dt=dt, **kwargs)
    return xs, cov

In [ ]:
P = np.diag([500., 49.])
Ms, Ps = run(count=50, R=10, Q=0.01, P=P)